In [1]:
import os
import sys
import yaml
import pickle
import glob
import copy

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import Parameters

from network_wrangler.utils import create_unique_shape_id

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

In [4]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
root_dir = "D:/github/lfs-cleaning/travel-model-two-networks"
input_dir = os.path.join(root_dir, 'data', 'processed')
output_dir = os.path.join(root_dir, 'data', 'processed', 'version_12')
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = "Z:/Data/Users/Sijia/MTC/github/Lasso"

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-09-09 12:06:59, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 12:06:59, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 12:06:59, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-09-09 12:06:59, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


In [6]:
version_11_pickle_file_name = os.path.join(
    input_dir, 
    'version_11',
    'working_scenario_01.pickle')
v_11_scenario = pickle.load(open(version_11_pickle_file_name, 'rb'))

In [7]:
version_12_pickle_file_name = os.path.join(
    input_dir, 
    'version_12', 
    'working_scenario_01.pickle')
v_12_scenario = pickle.load(open(version_12_pickle_file_name, 'rb'))

In [8]:
def create_ID_crosswalk(
    link_old,
    link_new,
    path
):
    """
    create the ID crosswalk between versions of network,
    in case the IDs are changed between version for the same geometries
    """
    
    link_old_df = link_old.copy()
    link_new_df = link_new.copy()

    
    link_old_df["shape_id"] = link_old_df["geometry"].apply(
                lambda x: create_unique_shape_id(x)
            )
    link_new_df["shape_id"] = link_new_df["geometry"].apply(
                lambda x: create_unique_shape_id(x)
            )
    link_join_df = pd.merge(
        link_old_df[["model_link_id", "A", "B","shape_id"]].rename(
            columns = {"model_link_id" : "model_link_id_old",
                       "A" : "A_old",
                       "B" : "B_old"}
        ),
        link_new_df[["model_link_id", "A", "B","shape_id"]].rename(
            columns = {"model_link_id" : "model_link_id_new",
                       "A" : "A_new",
                       "B" : "B_new"}
        ),
        how = "left",
        on = ["shape_id"]
    )
    
    link_join_df = link_join_df[
        (link_join_df.model_link_id_old != link_join_df.model_link_id_new) |
        (link_join_df.A_old != link_join_df.A_new) |
        (link_join_df.B_old != link_join_df.B_new)
    ]
    
    link_join_df[["model_link_id_old", "model_link_id_new"]].to_csv(
        os.path.join(path, "link_id_crosswalk.csv"),index=False)
    
    node_join_df = pd.concat(
        [link_join_df[["A_old", "A_new"]].rename(
            columns = {"A_old" : "model_node_id_old",
                       "A_new" : "model_node_id_new"}
        ),
         link_join_df[["B_old", "B_new"]].rename(
            columns = {"B_old" : "model_node_id_old",
                       "B_new" : "model_node_id_new"}
        )
        ],
        sort = False,
        ignore_index = True)
    
    node_join_df = node_join_df[
        node_join_df.model_node_id_old != node_join_df.model_node_id_new]
    node_join_df.drop_duplicates(inplace = True)
    
    node_join_df.to_csv(
        os.path.join(path, "node_id_crosswalk.csv"),index=False)
    
    return node_join_df,link_join_df

In [9]:
node_join_df, link_join_df = create_ID_crosswalk(
    link_old = v_11_scenario.road_net.links_df,
    link_new = v_12_scenario.road_net.links_df,
    path = output_dir
)

In [10]:
node_join_df[node_join_df.model_node_id_old == 1027709]

,model_node_id_old,model_node_id_new
95,1027709,1027710


# update project cards with new IDs

In [11]:
import glob

In [12]:
suffix = ["*.yaml", "*.yml", "*.wrangler", "*.wr"]
card_list = []
for s in suffix:
    card_list.extend(glob.glob(os.path.join(card_dir, s)))

In [13]:
len(card_list)

214

In [14]:
node_crosswalk = dict(
    zip(node_join_df.model_node_id_old, node_join_df.model_node_id_new)
)

link_crosswalk = dict(
    zip(link_join_df.model_link_id_old, link_join_df.model_link_id_new)
)

In [15]:
def map_new_id(x, crosswalk):
    return crosswalk.get(x)

def transit_id_change(d):
    for p in d.get('properties'):
        if p.get('property') == 'routing':
            _existing = []
            _set = []
            for n in p.get('existing'):
                if abs(n) in list(node_crosswalk.keys()):
                    if n > 0:
                        node = map_new_id(n, node_crosswalk)
                    else:
                        node = -(map_new_id(abs(n), node_crosswalk))
                else:
                    node = n
                _existing.append(node)
            for n in p.get('set'):
                if abs(n) in list(node_crosswalk.keys()):
                    if n > 0:
                        node = map_new_id(n, node_crosswalk)
                    else:
                        node = -(map_new_id(abs(n), node_crosswalk))
                else:
                    node = n
                _set.append(node)
                
            p['existing'] = _existing
            p['set'] = _set

    return d

def roadway_id_change(d):
    if d.get('category') in [
        'Roadway Property Change', 'Parallel Managed lanes'
    ]:

        for link in d.get('facility').get('link'):
            if "model_link_id" in list(link.keys()):
                
                _model_link_id_list = []
                for l in link.get('model_link_id'):
                    if l in list(link_crosswalk.keys()):
                        _model_link_id_list.append(
                            map_new_id(l, link_crosswalk)
                        )
                    else:
                        _model_link_id_list.append(
                            l
                        )

                link['model_link_id'] = _model_link_id_list
        
        return d
    
    if d.get('category') in ['Add New Roadway']:
        for link in d.get('links'):
            if link['A'] in list(node_crosswalk.keys()):
                link['A'] = map_new_id(link['A'], node_crosswalk)
            if link['B'] in list(node_crosswalk.keys()):
                link['B'] = map_new_id(link['B'], node_crosswalk)
        
        return d
    
    if d.get('category') in ['Roadway Deletion']:
        _model_link_id_list = []
        for l in d.get('links').get('model_link_id'):
            if l in list(link_crosswalk.keys()):
                link = map_new_id(l, link_crosswalk)

            else:
                link = l
            _model_link_id_list.append(link)

        d['links']['model_link_id'] = _model_link_id_list
        
        return d
        

def update_id(card):
    
    for change in card.changes:
        if change.get('category') == 'Transit Service Property Change':
            change = transit_id_change(change)

        else:
            change = roadway_id_change(change)

    return card

def update_project_cards(
    path,
):
    suffix = ["*.yaml", "*.yml"]
    card_list = []
    for s in suffix:
        card_list.extend(glob.glob(os.path.join(path, s)))
        
    for file in card_list:
        card = ProjectCard.read(
            file,
            validate = False
        )
        
        card = update_id(card)
        card.__dict__.pop('file', None)
        card.__dict__.pop('valid', None)
        
        card.write(
            out_filename = file
        )

In [16]:
update_project_cards(path = card_dir)

2021-09-09 12:12:53, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_new_roadway_antioch_bridge_CA-160_centroid_connector NB.yaml
2021-09-09 12:12:53, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_new_roadway_antioch_bridge_CA-160_centroid_connector NB.yaml
2021-09-09 12:12:53, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_new_roadway_antioch_bridge_CA-160_centroid_connector SB.yaml
2021-09-09 12:12:53, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_new_roadway_antioch_bridge_CA-160_centroid_connector SB.yaml
2021-09-09 12:12:53, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_new_roadway_cabrillo_highway_CA-1_centroid_connector NB.yaml
2021-09-09 12:12:53, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-

2021-09-09 12:12:53, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_new_roadway_vic_fazio_highway_CA-113_centroid_connector SB.yaml
2021-09-09 12:12:53, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_new_roadway_viz_fazio_highway_CA-113_centroid_connector NB.yaml
2021-09-09 12:12:53, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_new_roadway_viz_fazio_highway_CA-113_centroid_connector NB.yaml
2021-09-09 12:12:53, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_new_roadway_william_elton_brown_freeway_I-580_centroid_connector NB.yaml
2021-09-09 12:12:53, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_new_roadway_william_elton_brown_freeway_I-580_centroid_connector NB.yaml
2021-09-09 12:12:53, INFO: Wrote project card t

2021-09-09 12:12:55, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\sfmta-route-381-edits.yml
2021-09-09 12:12:55, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\SFMTA_14.yml
2021-09-09 12:12:55, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\SFMTA_14.yml
2021-09-09 12:12:55, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\SFMTA_14R.yml
2021-09-09 12:12:55, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\SFMTA_14R.yml
2021-09-09 12:12:55, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\SFMTA_22.yml
2021-09-09 12:12:55, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\SFMTA_22.yml
2021-09-09 12:12:55, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\proj

2021-09-09 12:12:58, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_attributes_exclude_trucks_sr85.yml
2021-09-09 12:12:58, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_attributes_exclude_trucks_sr85.yml
2021-09-09 12:12:58, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_bus_only_ggt_on_us101.yml
2021-09-09 12:12:58, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_bus_only_ggt_on_us101.yml
2021-09-09 12:12:58, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_bus_only_transbay_terminal_ramps.yml
2021-09-09 12:12:58, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_bus_only_transbay_terminal_ramps.yml
2021-09-09 12:12:58, INFO: Wrote project card to: D:/github/lfs-cl

2021-09-09 12:12:58, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_managed_lane_e_capitol_expresswayn_us101_to_i680.yml
2021-09-09 12:12:58, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_managed_lane_e_capitol_expresswayn_us101_to_i680.yml
2021-09-09 12:12:58, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_managed_lane_e_capitol_expressways_i680_to_us101.yml
2021-09-09 12:12:58, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_managed_lane_e_capitol_expressways_i680_to_us101.yml
2021-09-09 12:12:58, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_managed_lane_hwy4e_port_chicago_highway_to_deer_valley_road.yml
2021-09-09 12:12:58, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_ca

2021-09-09 12:12:58, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_managed_lane_i80e_i-680_to_fairfield.yml
2021-09-09 12:12:58, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_managed_lane_i80w_carquinez_bridge_to_bay_bridge.yml
2021-09-09 12:12:58, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_managed_lane_i80w_carquinez_bridge_to_bay_bridge.yml
2021-09-09 12:12:59, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_managed_lane_i80w_fairfield_to_i680.yml
2021-09-09 12:12:59, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_managed_lane_i80w_fairfield_to_i680.yml
2021-09-09 12:12:59, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_managed_lane_i80_ramp_to_bay_bridge

2021-09-09 12:12:59, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_managed_lane_us101s_segment02_south_healdsburg_to_north_petaluma.yml
2021-09-09 12:12:59, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_pacheco_pass_highway_missing_segments.yml
2021-09-09 12:12:59, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_pacheco_pass_highway_missing_segments.yml
2021-09-09 12:12:59, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_richmond_bridge_toll_plaza.yml
2021-09-09 12:12:59, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_richmond_bridge_toll_plaza.yml
2021-09-09 12:12:59, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_roadway_property_change_benicia_bridge_segment_

2021-09-09 12:13:00, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_roadway_property_change_should_be_3_lane_should_be_expressway.yml
2021-09-09 12:13:00, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_roadway_property_change_should_be_3_lane_should_be_expressway.yml
2021-09-09 12:13:00, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_roadway_property_change_should_be_4_lanes.yml
2021-09-09 12:13:00, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_roadway_property_change_should_be_4_lanes.yml
2021-09-09 12:13:00, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_roadway_property_change_should_be_4_lane_correct_facility_type.yml
2021-09-09 12:13:00, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-netw

2021-09-09 12:13:00, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_san_mateo_bridge_toll_plaza.yml
2021-09-09 12:13:00, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_san_mateo_bridge_toll_plaza.yml
2021-09-09 12:13:00, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_san_mateo_bridge_toll_plaza_transit.yml
2021-09-09 12:13:00, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_san_mateo_bridge_toll_plaza_transit.yml
2021-09-09 12:13:00, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_service_road_carquinez_bridge_bypass.yml
2021-09-09 12:13:00, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\project_cards\year_2015_service_road_carquinez_bridge_bypass.yml
2021-09-09 12:13:00, INFO: Wrote pro

In [158]:
card = ProjectCard.read(
    r"D:\github\lfs-cleaning\travel-model-two-networks\project_cards\year_2015_san_mateo_bridge_add_plaza_link.yml",
    validate = False
)

In [161]:
card.changes

[{'category': 'Roadway Deletion', 'links': {'model_link_id': [3073690]}},
 {'category': 'Add New Roadway',
  'links': [{'A': 2562924,
    'B': 5030000,
    'bike_access': 0,
    'county': 'Alameda',
    'drive_access': 1,
    'model_link_id': 3315462,
    'rail_only': 0,
    'shstGeometryId': '',
    'walk_access': 0,
    'tollbooth': 0,
    'ft': 1,
    'lanes': 4,
    'assignable': 1},
   {'A': 5030000,
    'B': 2535792,
    'bike_access': 0,
    'county': 'Alameda',
    'drive_access': 1,
    'model_link_id': 3315463,
    'rail_only': 0,
    'shstGeometryId': '',
    'walk_access': 0,
    'ft': 7,
    'lanes': 7,
    'assignable': 1}],
  'nodes': [{'bike_access': 0,
    'county': 'Alameda',
    'drive_access': 1,
    'rail_only': 0,
    'walk_access': 0,
    'model_node_id': 5030000,
    'osm_node_id': '',
    'X': -122.147744,
    'Y': 37.6193308}]}]

In [166]:
new_card = update_id(card)

In [168]:
print(new_card)

project: add san mateo bridge toll plaza
tags: ['Year 2015', 'Toll Plaza']
dependencies: 
changes: [{'category': 'Roadway Deletion', 'links': {'model_link_id': ['test']}}, {'category': 'Add New Roadway', 'links': [{'A': 'test', 'B': 5030000, 'bike_access': 0, 'county': 'Alameda', 'drive_access': 1, 'model_link_id': 3315462, 'rail_only': 0, 'shstGeometryId': '', 'walk_access': 0, 'tollbooth': 0, 'ft': 1, 'lanes': 4, 'assignable': 1}, {'A': 5030000, 'B': 2535792, 'bike_access': 0, 'county': 'Alameda', 'drive_access': 1, 'model_link_id': 3315463, 'rail_only': 0, 'shstGeometryId': '', 'walk_access': 0, 'ft': 7, 'lanes': 7, 'assignable': 1}], 'nodes': [{'bike_access': 0, 'county': 'Alameda', 'drive_access': 1, 'rail_only': 0, 'walk_access': 0, 'model_node_id': 5030000, 'osm_node_id': '', 'X': -122.147744, 'Y': 37.6193308}]}]
valid: False
file: D:\github\lfs-cleaning\travel-model-two-networks\project_cards\year_2015_san_mateo_bridge_add_plaza_link.yml


In [169]:
new_card.write(out_filename = os.path.join(output_dir, "test.yml"))

2021-09-02 17:32:46, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_12\test.yml
2021-09-02 17:32:46, INFO: Wrote project card to: D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_12\test.yml


In [190]:
node_join_df[node_join_df.model_node_id_new == 1028039]

,model_node_id_old,model_node_id_new
550,1028038,1028039
